In [1]:
from __future__ import division

from glob import glob
import os
import os.path
import re
import pandas as pd
import csv
import tgt #TextGridTools by Buschmeier and Wlodarczak
import pickle
from fnmatch import fnmatch
from itertools import cycle

In [4]:
rootdir = '/Users/malisz/Corpora/EMU_Studio_POL/VOT'

In [5]:
#TABELA DO ANALIZY
for dirs, subdir, files in os.walk(rootdir):

    files = [f for f in files if not f[0] == '.']
    subdir[:] = [d for d in subdir if not d[0] == '.']
    
    txgr = "*.TextGrid"
    for filename in files:
        if fnmatch(filename, txgr):
            
            basename = filename[:-9]
            path = os.path.join(dirs, filename)
            print('Processing file' + path)
            
            t = tgt.read_textgrid(path, encoding='utf-8')
            
            seg = t.get_tier_by_name('segments')
            vless = t.get_tier_by_name('vot')
            voiced = t.get_tier_by_name('votvoiced')
    
    #create new tiers
            clos1=tgt.IntervalTier(name='closure_voiceless')
            clos2=tgt.IntervalTier(name='closure_voiced')
    
    #find the closure phase of the stop        
            def closure(closure_tier_name, segment_tier_name, vot_tier_name):
                clos_start = []
                clos_end = []
                for i in segment_tier_name.intervals:
                    seg_start = i.start_time
                    seg_end = i.end_time
                    for j in vot_tier_name.intervals:
                        if (j.start_time >= seg_start) and (j.start_time <= seg_end):
                            clos_start.append(seg_start)
                            clos_end.append(j.start_time)
                for i, j in zip(clos_start, clos_end):
                    closure_tier_name.add_interval((tgt.Interval(start_time=i, end_time=j, text='closure')))
                return closure_tier_name
            
            clos1=closure(clos1, seg, vless)
            clos2=closure(clos2, seg, voiced)

            t.add_tier(clos1)
            t.add_tier(clos2)
            path1 = os.path.join(dirs, basename)
            #tgt.write_to_file(t, path1 + '_test' + '.TextGrid', format='long', encoding='utf-8')
            
    #prepare the Windows tier from segments tier and check if the next interval after the stop is a vowel.

            seg2=tgt.IntervalTier(name='windows')
        
            stops_PL = ['p', 't', 'k', 'b', 'd', 'g']
            #not sure if en is the nasal e
            vowels_PL = ['a', 'e', 'i', 'o', 'u', 'on', 'I', 'en']
            
            win_start = []
            win_end = []
            label = []
            
            for idx, elem in enumerate(seg.intervals):

                thiselem = elem
                nextelem = seg.intervals[(idx + 1) % len(seg.intervals)]
                if thiselem.text in stops_PL and nextelem.text in vowels_PL:
                    win_start.append(thiselem.start_time - 0.03)
                    win_end.append(thiselem.end_time + 0.03)
                    label.append(thiselem.text)
            
            for i, j in zip(win_start, win_end):
                try:
                    seg2.add_interval((tgt.Interval(start_time=i, end_time=j, text='w')))
                except ValueError:
                    pass
            
            t.add_tier(seg2)
            tgt.write_to_file(t, path1 + '_windows' + '.TextGrid', format='long', encoding='utf-8')
           
    
 
            

Processing file/Users/malisz/Corpora/EMU_Studio_POL/VOT/SES0001/sent002.TextGrid
Processing file/Users/malisz/Corpora/EMU_Studio_POL/VOT/SES0001/sent003.TextGrid
Processing file/Users/malisz/Corpora/EMU_Studio_POL/VOT/SES0001/sent004.TextGrid
Processing file/Users/malisz/Corpora/EMU_Studio_POL/VOT/SES0001/sent005.TextGrid
Processing file/Users/malisz/Corpora/EMU_Studio_POL/VOT/SES0001/sent006.TextGrid
Processing file/Users/malisz/Corpora/EMU_Studio_POL/VOT/SES0001/sent026.TextGrid
Processing file/Users/malisz/Corpora/EMU_Studio_POL/VOT/SES0001/sent027.TextGrid
Processing file/Users/malisz/Corpora/EMU_Studio_POL/VOT/SES0001/sent028.TextGrid
Processing file/Users/malisz/Corpora/EMU_Studio_POL/VOT/SES0001/sent029.TextGrid
Processing file/Users/malisz/Corpora/EMU_Studio_POL/VOT/SES0001/sent030.TextGrid
Processing file/Users/malisz/Corpora/EMU_Studio_POL/VOT/SES0002/sent001.TextGrid
Processing file/Users/malisz/Corpora/EMU_Studio_POL/VOT/SES0002/sent002.TextGrid
Processing file/Users/malisz